<a href="https://colab.research.google.com/github/srujan-b/World-best-XI-from-Mens-ICC-world-cup-2023-against-australia/blob/main/webScrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from bs4 import BeautifulSoup
import requests
import smtplib
import time
from datetime import datetime
import re

In [3]:
# Url defination
url = 'https://www.espncricinfo.com/series/icc-cricket-world-cup-2023-24-1367856/match-schedule-fixtures-and-results'

# Header for web scrapping
headers = {"User-Agen":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 Brave/96.0.4664.110","Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate, br, zstd", "Accept-Language": "en-US,en","DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}



page = requests.get(url,headers = headers)

In [4]:
resultPage = BeautifulSoup(page.content, "html.parser")

In [5]:
# Extracting match Dates

dates = resultPage.find_all("div", {"class": "ds-text-compact-xs ds-font-bold ds-w-24"})

date = []
for i in dates:

  if i.get_text():
    date_string = i.get_text(strip=True).split(',')[1]
    date_object = datetime.strptime(date_string, " %d %b '%y")
    formatted_date = date_object.strftime("%Y-%m-%d")
    date.append(formatted_date)
  else:
    date.append(formatted_date)

In [6]:
# Extracting match numbers

matchNumbers = resultPage.find_all("span", {"class": "ds-text-tight-s ds-font-medium ds-text-typo"})

matchNum = []
for i in matchNumbers:
  matchDetails = i.get_text(strip =True).split(",")[0].split()
  if matchDetails[1] == 'Match':
    pattern = r'\d+'
    numbers = int(re.search(pattern, matchDetails[0]).group())
    matchNum.append(numbers)
  else:
    matchNum.append(" ".join(matchDetails[0:-1]))

In [7]:
# Extracting City

place = resultPage.find_all("div", {"class": "ds-text-tight-s ds-font-regular ds-truncate ds-text-typo-mid3"})

city = []

for i in place:
  if i.contents[3]:
    city.append(i.contents[3])


In [8]:
# extracting teams
team1Find = resultPage.find_all("p", {"class": "ds-text-tight-m ds-font-bold ds-capitalize ds-truncate"})

teamName = {'England':'ENG', 'India':'IND', 'Scotland':'SCOT', 'Ireland':'IRE', 'Sri Lanka':'SL', 'Zimbabwe':'ZIM', 'Afghanistan':'AFG', 'Australia':'AUS', 'Pakistan':'PAK', 'United Arab Emirates':'UAE', 'Bangladesh':'BAN', 'South Africa':'SA', 'Netherlands':'NED', 'New Zealand':'NZ', 'Namibia':'NAM', 'West Indies':'WI'}
team1 = []
team2 = []
team1SF = []
team2SF = []
a = 1
for i in team1Find:
  if a % 2 != 1:
    team = i.get_text(strip=True)
    team1.append(team)
    team1SF.append(teamName[team])
    a +=1
  else:
    team = i.get_text(strip=True)
    team2.append(team)
    team2SF.append(teamName[team])
    a+=1


In [9]:
# scores of the match

scores_class_divs = resultPage.find_all("div", {"class": "ds-flex ds-flex-col ds-mt-2 ds-mb-2"})

team1Score = []
team2Score = []

a = 1
for div in scores_class_divs:

  if div.find_all("strong", class_=""):
    bothTeamScore = div.find_all("strong", class_="")
    team1s = bothTeamScore[0].get_text(strip=True)
    team2s = bothTeamScore[1].get_text(strip=True)
    team1Score.append(team1s)
    team2Score.append(team2s)
  else:
    team1Score.append(" ")
    team2Score.append(" ")




In [10]:
# Results

resultsClassDivs = resultPage.find_all("p", {"class": "ds-text-tight-s ds-font-medium ds-line-clamp-2 ds-text-typo"})

results = []


for i in resultsClassDivs:
  results.append(i.find("span").get_text(strip=True))



In [11]:
#scorecardURl

scoreCardUrlDivs = resultPage.find_all("a", {"class": "ds-no-tap-higlight"})

url = []
for i in scoreCardUrlDivs:

  url.append("https://www.espncricinfo.com"+i.get("href"))


In [12]:
import pandas as pd
finalList = zip(date,matchNum,city,team1,team1SF,team1Score,team2,team2SF,team2Score,results,url)

df = pd.DataFrame(finalList, columns=('MatchDate','MatchNumber','City','Team1','Team1SF','Team1Score','Team2','Team2SF','Team2Score','Results','url'))


In [13]:
df

,MatchDate,MatchNumber,City,Team1,Team1SF,Team1Score,Team2,Team2SF,Team2Score,Results,url
0,2023-10-05,1,Ahmedabad,New Zealand,NZ,282/9,England,ENG,283/1,New Zealand won by 9 wickets (with 82 balls re...,https://www.espncricinfo.com/series/indian-pre...
1,2023-10-06,2,Hyderabad,Netherlands,NED,286,Pakistan,PAK,205,Pakistan won by 81 runs,https://www.espncricinfo.com/series/usa-vs-can...
2,2023-10-07,3,Dharamsala,Bangladesh,BAN,156,Afghanistan,AFG,158/4,Bangladesh won by 6 wickets (with 92 balls rem...,https://www.espncricinfo.com/series/indian-pre...
3,2023-10-07,4,Delhi,Sri Lanka,SL,428/5,South Africa,SA,326,South Africa won by 102 runs,https://www.espncricinfo.com/series/west-indie...
4,2023-10-08,5,Chennai,India,IND,199,Australia,AUS,201/4,India won by 6 wickets (with 52 balls remaining),https://www.espncricinfo.com/series/west-indie...
5,2023-10-09,6,Hyderabad,Netherlands,NED,322/7,New Zealand,NZ,223,New Zealand won by 99 runs,https://www.espncricinfo.com/series/west-indie...
6,2023-10-10,7,Dharamsala,Bangladesh,BAN,364/9,England,ENG,227,England won by 137 runs,https://www.espncricinfo.com/series/west-indie...
7,2023-10-10,8,Hyderabad,Pakistan,PAK,344/9,Sri Lanka,SL,345/4,Pakistan won by 6 wickets (with 10 balls remai...,https://www.espncricinfo.com/series/icc-cricke...
8,2023-10-11,9,Delhi,India,IND,272/8,Afghanistan,AFG,273/2,India won by 8 wickets (with 90 balls remaining),https://www.espncricinfo.com/series/icc-cricke...
9,2023-10-12,10,Lucknow,Australia,AUS,311/7,South Africa,SA,177,South Africa won by 134 runs,https://www.espncricinfo.com/series/icc-cricke...


In [174]:
for i in zip(date,matchNum,city,team1,team1SF,team1Score,team2,team2SF,team2Score,results,url):

  print(i)

('2022-10-16', 1, 'Geelong', 'Sri Lanka', 'SL', '163/7', 'Namibia', 'NAM', '108', 'Namibia won by 55 runs', 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/rajasthan-royals-vs-gujarat-titans-24th-match-1426262/live-cricket-score')
('2022-10-16', 2, 'Geelong', 'Netherlands', 'NED', '111/8', 'United Arab Emirates', 'UAE', '112/7', 'Netherlands won by 3 wickets (with 1 ball remaining)', 'https://www.espncricinfo.com/series/usa-vs-canada-2024-1425115/united-states-of-america-vs-canada-3rd-t20i-1425123/live-cricket-score')
('2022-10-17', 3, 'Hobart', 'West Indies', 'WI', '160/5', 'Scotland', 'SCOT', '118', 'Scotland won by 42 runs', 'https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/mumbai-indians-vs-royal-challengers-bengaluru-25th-match-1426263/live-cricket-score')
('2022-10-17', 4, 'Hobart', 'Ireland', 'IRE', '174/7', 'Zimbabwe', 'ZIM', '143/9', 'Zimbabwe won by 31 runs', 'https://www.espncricinfo.com/series/icc-men-s-t20-world-cup-2022-23-129